In [1]:
## import essential modules 
import qumcmc 
from qumcmc.basic_utils import *
from qumcmc.energy_models import IsingEnergyFunction, Exact_Sampling

from qumcmc.classical_mcmc_routines import classical_mcmc
# from qumcmc.quantum_mcmc_routines_qiskit import quantum_enhanced_mcmc   #for qiskit Aer's Simulator backend 
from qumcmc.quantum_mcmc_routines_qulacs import quantum_enhanced_mcmc   #for qulacs backend (**faster)

from qumcmc.trajectory_processing import calculate_running_js_divergence, calculate_running_kl_divergence, calculate_runnning_magnetisation, get_trajectory_statistics

from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

In [2]:
import pickle
import seaborn as sns
from matplotlib.colors import LogNorm, Normalize

In [72]:
from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, transpiler, execute, Aer
from qiskit.visualization import plot_circuit_layout, plot_coupling_map
# provider = IBMProvider(token= '0c4a11984552ee16d40f9f7ad8848d8393c9010d55e924306fbc7486c2c55927173b79841d947e255fdbc927079265ed02aa66f10895a50367a0fd97bc6de628')


In [25]:
lima = provider.get_backend('ibmq_lima'); guadalupe= provider.get_backend('ibmq_guadalupe')

In [77]:
lima_cnfg = lima.configuration()

In [81]:
plot_coupling_map(lima_cnfg)

TypeError: plot_coupling_map() missing 2 required positional arguments: 'qubit_coordinates' and 'coupling_map'

In [75]:
from qiskit.providers.models import backendconfiguration

In [87]:
lima_cnfg.coupling_map

[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 4], [4, 3]]

In [84]:
cmp = transpiler.CouplingMap(lima_cnfg.coupling_map , 'tri')

In [95]:
list(cmp.get_edges())

[(0, 1), (1, 0), (1, 2), (1, 3), (2, 1), (3, 1), (3, 4), (4, 3)]

In [28]:
theta= np.random.uniform(0, 2 * np.pi)

In [55]:
qc = QuantumCircuit(2)

qc.h(0); qc.h(1)

for _ in range(10):
    qc.h(1)
    qc.rzx(theta, 0, 1)
    qc.h(1)

qc.measure_all()

tjb = transpile(qc, lima, initial_layout= [1,3])
# plot_circuit_layout(tjb, lima)
ec = execute(tjb, lima)
ec.result().get_counts()

{'00': 989, '01': 915, '10': 1091, '11': 1005}

In [56]:
qc_ = QuantumCircuit(2)

qc_.h(0); qc_.h(1)
for _ in range(10):    
    qc_.rzz(theta, 0, 1)

qc_.measure_all()

tjb = transpile(qc_, lima, initial_layout= [1,3])
# plot_circuit_layout(tjb, lima)

ec_ = execute(tjb, lima)
ec_.result().get_counts()

{'00': 1004, '01': 909, '10': 1138, '11': 949}

In [65]:
ecs = execute(qc_, Aer.get_backend('aer_simulator'), shots= 4000)
ecs.result().get_counts()

{'11': 1003, '10': 1007, '00': 1022, '01': 968}

In [66]:
ec_.result().time_taken, ec.result().time_taken, ecs.result().time_taken

(2.8768982887268066, 2.9189612865448, 0.002936124801635742)